In [7]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SQLContext

In [5]:
user = "postgres"
password = "Felipe1998"
hostname = "localhost"
db_name = "postgres"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{hostname}/{db_name}")

In [14]:
create_table_query ="""

    DROP TABLE IF EXISTS pyspark_tables.fire_service_calls CASCADE;

    CREATE TABLE pyspark_tables.fire_service_calls(
        call_number INTEGER,
        unit_id TEXT,
        incident_number INTEGER NOT NULL,
        call_type TEXT,
        call_date TIMESTAMP,
        watch_date TIMESTAMP,
        received_dttm TIMESTAMP,
        entry_dttm TIMESTAMP,
        dispatch_dttm TIMESTAMP,
        response_dttm TIMESTAMP,
        on_scene_dttm TIMESTAMP,
        transport_dttm TIMESTAMP,
        hospital_dttm TIMESTAMP,
        call_final_disposition TEXT,
        available_dttm TIMESTAMP,
        address TEXT,
        city TEXT,
        zipcode_of_incident INTEGER,
        battalion TEXT,
        station_area INTEGER,
        box INTEGER,
        original_priority TEXT,
        priority TEXT,
        final_priority INTEGER,
        als_unit BOOLEAN,
        call_type_group TEXT,
        number_of_alarms INTEGER,
        unit_type TEXT,
        unit_sequence_in_call_dispatch INTEGER,
        fire_prevention_district TEXT,
        supervisor_district TEXT,
        neighborhoods_analysis_boundaries TEXT,
        rowid TEXT,
        case_location TEXT,
        data_as_of TIMESTAMP,
        data_loaded_at TIMESTAMP,
        computed_region_ajp5_b2md INTEGER,

        PRIMARY KEY(incident_number)
    );

"""

engine.execute(create_table_query)

In [15]:
#spark session
spark = SparkSession.builder.appName("fire_calls").getOrCreate()
#sql context
sqlcontext = SQLContext(spark)

In [16]:
#reading a csv file.
sf_fire_calls_df = spark.read.csv("../data/fire_department_calls.csv", header=True, inferSchema=True)